# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from sklearn.model_selection import train_test_split

# Import API key
from api_keys import g_key

#configure api key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#define the file path for the output data csv in part 1
file_path = os.path.join("../output_data/weather.csv")

#read in the csv into a dataframe
weather_df = pd.read_csv(file_path)

weather_df.head()


,City,Country,Longitude,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,zhezkazgan,KZ,67.7144,47.8043,12.20,94,0,7.65
1,avarua,CK,-159.7750,-21.2078,86.00,70,33,14.97
2,arraial do cabo,BR,-42.0278,-22.9661,83.59,65,0,11.50
3,liliani,PK,72.9537,32.2065,47.30,47,0,2.28
4,cherskiy,RU,161.3000,68.7500,-28.43,85,98,2.21


In [3]:
#trim data to a test dataset for testing code 
#("test" will be the test dataset I use which will have to be replaced with weather_df at the end)
# train, test = train_test_split(weather_df, test_size=0.01, random_state=42, shuffle=True)
# test
    
    

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#establish the map figure in the notebook
fig = gmaps.figure()

In [5]:
#create lists for locations and weights for the heatmap
locations = weather_df[["Latitude", "Longitude"]]
locations



,Latitude,Longitude
0,47.8043,67.7144
1,-21.2078,-159.7750
2,-22.9661,-42.0278
3,32.2065,72.9537
4,68.7500,161.3000
...,...,...
562,42.4408,11.2196
563,13.2019,-16.7339
564,39.1834,-84.5333
565,-36.8927,-60.3225


In [6]:
humidity = weather_df["Humidity (%)"].astype(float)
humidity

0      94.0
1      70.0
2      65.0
3      47.0
4      85.0
       ... 
562    87.0
563    44.0
564    51.0
565    15.0
566    80.0
Name: Humidity (%), Length: 567, dtype: float64

In [10]:
#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)

#add heat layer to fig
fig.add_layer(heat_layer)

#display fig
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#apply conditionals to the weather_df so that we have a new dataframe with ideal weather condition cities
hotel_df = weather_df.loc[(weather_df["Temperature (F)"] > 65) & (weather_df["Temperature (F)"] < 75) 
                          & (weather_df["Wind Speed (mph)"] < 8) & (weather_df["Humidity (%)"] < 70) 
                          & (weather_df["Cloudiness (%)"] < 40)]
hotel_df = pd.DataFrame(hotel_df)
hotel_df

,City,Country,Longitude,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
22,myingyan,MM,95.3833,21.4667,67.19,59,0,3.78
49,guerrero negro,MX,-114.0611,27.9769,69.98,27,0,6.11
93,camana,PE,-72.7111,-16.6228,74.80,61,6,7.25
118,asosa,ET,34.5333,10.0667,69.93,27,0,5.44
205,adelaide,AU,138.6000,-34.9333,66.61,68,0,3.44
218,sao filipe,CV,-24.4956,14.8961,74.88,67,0,4.03
221,taltal,CL,-70.4833,-25.4000,70.09,62,24,6.13
276,saint-leu,RE,55.2833,-21.1500,70.72,61,8,3.74
279,coromoro,CO,-73.0402,6.2946,71.74,67,22,3.60
303,odweyne,SO,45.0640,9.4092,65.88,50,0,1.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#add hotel name column to dataframe
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Longitude,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
22,myingyan,MM,95.3833,21.4667,67.19,59,0,3.78,
49,guerrero negro,MX,-114.0611,27.9769,69.98,27,0,6.11,
93,camana,PE,-72.7111,-16.6228,74.80,61,6,7.25,
118,asosa,ET,34.5333,10.0667,69.93,27,0,5.44,
205,adelaide,AU,138.6000,-34.9333,66.61,68,0,3.44,
218,sao filipe,CV,-24.4956,14.8961,74.88,67,0,4.03,
221,taltal,CL,-70.4833,-25.4000,70.09,62,24,6.13,
276,saint-leu,RE,55.2833,-21.1500,70.72,61,8,3.74,
279,coromoro,CO,-73.0402,6.2946,71.74,67,22,3.60,
303,odweyne,SO,45.0640,9.4092,65.88,50,0,1.92,


In [13]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
    "location": "6.2946,-73.0402"
}

In [15]:
#create empty list for hotel names
hotels = []

#loop through the dataframe to get the lat/lng of each location to search for nearby hotels
for index, row in hotel_df.iterrows():
    
    try:
        #get lat, lng from df
        lat = row["Latitude"]
        lng = row["Longitude"]

        #add latitude and longitude to params
        params["location"] = f"{lat},{lng}"

        #establish the base url for Google maps nearby search
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        #make request for url
        response = requests.get(base_url, params=params)

        #jsonify the response
        json = response.json()

        #add selected hotel to the hotel list
        hotels.append(json["results"][0]["name"])
        
        #add hotels to column in dataframe
        
        
    
    except:
        print(f"no hotel found within the selected radius for {row['City']}")
        hotels.append("N/A")

no hotel found within the selected radius for coromoro
no hotel found within the selected radius for kurgunta


In [16]:
#create new column in the df called Hotel Name
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Longitude,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
22,myingyan,MM,95.3833,21.4667,67.19,59,0,3.78,Nan Myint Hotel (Myingyan)
49,guerrero negro,MX,-114.0611,27.9769,69.98,27,0,6.11,Hotel TerraSal
93,camana,PE,-72.7111,-16.6228,74.80,61,6,7.25,Hotel de Turistas
118,asosa,ET,34.5333,10.0667,69.93,27,0,5.44,Bamboo Paradise Hotel
205,adelaide,AU,138.6000,-34.9333,66.61,68,0,3.44,Crowne Plaza Adelaide
218,sao filipe,CV,-24.4956,14.8961,74.88,67,0,4.03,Hotel Xaguate
221,taltal,CL,-70.4833,-25.4000,70.09,62,24,6.13,"Hotel- Galería ""Las brisas"""
276,saint-leu,RE,55.2833,-21.1500,70.72,61,8,3.74,Iloha Seaview Hotel
279,coromoro,CO,-73.0402,6.2946,71.74,67,22,3.60,N/A
303,odweyne,SO,45.0640,9.4092,65.88,50,0,1.92,Al shukri hotel


In [17]:
#drop the N/A hotel names
hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name']=="N/A"].index, inplace=True)

hotel_df

,City,Country,Longitude,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
22,myingyan,MM,95.3833,21.4667,67.19,59,0,3.78,Nan Myint Hotel (Myingyan)
49,guerrero negro,MX,-114.0611,27.9769,69.98,27,0,6.11,Hotel TerraSal
93,camana,PE,-72.7111,-16.6228,74.80,61,6,7.25,Hotel de Turistas
118,asosa,ET,34.5333,10.0667,69.93,27,0,5.44,Bamboo Paradise Hotel
205,adelaide,AU,138.6000,-34.9333,66.61,68,0,3.44,Crowne Plaza Adelaide
218,sao filipe,CV,-24.4956,14.8961,74.88,67,0,4.03,Hotel Xaguate
221,taltal,CL,-70.4833,-25.4000,70.09,62,24,6.13,"Hotel- Galería ""Las brisas"""
276,saint-leu,RE,55.2833,-21.1500,70.72,61,8,3.74,Iloha Seaview Hotel
303,odweyne,SO,45.0640,9.4092,65.88,50,0,1.92,Al shukri hotel
558,nouadhibou,MR,-17.0347,20.9310,73.40,56,0,6.91,HOtel Tasiast


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
for index, row in hotel_df.iterrows():
    hotel_info = info_box_template.format(**row)
    marker_locations = hotel_df[["Latitude", "Longitude"]]
    markers = gmaps.marker_layer(marker_locations, info_box_content=f"Hotel Info: {hotel_info}")

In [19]:
# Add marker layer ontop of heat map
fig.add_layer(markers)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))